# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [71]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, count, avg
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.functions import isnan

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify Project") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
path = "mini_sparkify_event_data.json"
df = spark.read.json(path)

In [4]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
df.createOrReplaceTempView("df_table")

In [6]:
spark.sql('''
            SELECT *
            FROM df_table
            WHERE userId == '132'
          ''').show(1)

+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+-----------------+------+-------------+--------------------+------+
|           artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|             song|status|           ts|           userAgent|userId|
+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+-----------------+------+-------------+--------------------+------+
|Justin Timberlake|Logged In|    Sadie|     F|            0|   Jones|275.66975| free|Denver-Aurora-Lak...|   PUT|NextSong|1537054553000|      131|Still On My Brain|   200|1538470796000|"Mozilla/5.0 (Mac...|   132|
+-----------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+--------

In [7]:
spark.sql('''
            SELECT *
            FROM df_table
            WHERE page == 'Cancellation Confirmation'
          ''').show(1)

+------+---------+---------+------+-------------+--------+------+-----+------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|          location|method|                page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+--------+------+-----+------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Cancelled|   Adriel|     M|          104| Mendoza|  null| paid|Kansas City, MO-KS|   GET|Cancellation Conf...|1535623466000|      514|null|   200|1538943990000|"Mozilla/5.0 (Mac...|    18|
+------+---------+---------+------+-------------+--------+------+-----+------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
only showing to

In [8]:
df.select('gender').distinct().show()
df.select('page').distinct().show(200)

+------+
|gender|
+------+
|     F|
|  null|
|     M|
+------+

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+



In [9]:
df.select('status').distinct().show()
df.select('userId').distinct().show(5)

+------+
|status|
+------+
|   307|
|   404|
|   200|
+------+

+------+
|userId|
+------+
|100010|
|200002|
|   125|
|    51|
|   124|
+------+
only showing top 5 rows



In [10]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [11]:
print((df.count(), len(df.columns)))

(286500, 18)


In [12]:
import pyspark.sql.functions as F

df_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])

In [13]:
df_agg.show()

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId| song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+
| 58392|   0|     8346|  8346|            0|    8346| 58392|    0|    8346|     0|   0|        8346|        0|58392|     0|  0|     8346|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+-----+------+---+---------+------+



In [14]:
df_valid = df.dropna(how='any', subset = ['userId', 'sessionId'])

In [15]:
print(df.count())
print(df_valid.count())

286500
286500


In [16]:
df_valid = df.filter(df['userId'] != '')
print(df_valid.count())

278154


In [17]:
df = df_valid

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [18]:
df.filter("page = 'Cancellation Confirmation'").show(5)

+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|artist|     auth|firstName|gender|itemInSession|lastName|length|level|            location|method|                page| registration|sessionId|song|status|           ts|           userAgent|userId|
+------+---------+---------+------+-------------+--------+------+-----+--------------------+------+--------------------+-------------+---------+----+------+-------------+--------------------+------+
|  null|Cancelled|   Adriel|     M|          104| Mendoza|  null| paid|  Kansas City, MO-KS|   GET|Cancellation Conf...|1535623466000|      514|null|   200|1538943990000|"Mozilla/5.0 (Mac...|    18|
|  null|Cancelled|    Diego|     M|           56|   Mckee|  null| paid|Phoenix-Mesa-Scot...|   GET|Cancellation Conf...|1537167593000|      540|null|   200|1539033046000|"Mozilla/5.0 (iPh...|    32|
|  nu

In [19]:
df.select(['userId', 'gender', 'level', 'location', 'ts', 'artist', 'page']).where(df.userId == '18').collect()

[Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538499917000, artist=None, page='Home'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538499933000, artist='Mike And The Mechanics', page='NextSong'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538500208000, artist='Taking Back Sunday', page='NextSong'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538500476000, artist='Beirut', page='NextSong'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538500654000, artist='Bob Log III', page='NextSong'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538500842000, artist='Krisiun', page='NextSong'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538500856000, artist=None, page='Settings'),
 Row(userId='18', gender='M', level='paid', location='Kansas City, MO-KS', ts=1538

In [20]:
df.select(['page']).where(df.userId == '18').collect()

[Row(page='Home'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Logout'),
 Row(page='Home'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='Add to Playlist'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='NextSong'),
 Row(page='N

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

## Features

### Inputs
Thumbs Up total
Thumbs Down total
Add to Playlist total
Add Friend total
Error total
Next Song count per day for last 7 days
Next Song count per hour on average over all time

### Prediction
Churn 0 or 1


In [21]:
# get list of users
users = df.select('userId').distinct().toPandas()
print(users.count())
users.head()

userId    225
dtype: int64


,userId
0,100010
1,200002
2,125
3,51
4,124


In [22]:
# get list of page counts overall
total_counts = df.groupBy('userId').agg(count('*').alias('count')).sort('userId').toPandas()
total_counts.set_index('userId', inplace=True)
total_counts.head()

,count
userId,
10,795
100,3214
100001,187
100002,218
100003,78


In [23]:
# get list of page counts by page type
page_counts = df.groupBy('userId', 'page').agg(count('*').alias('count')).sort('userId', 'page').toPandas()
page_counts.head()

,userId,page,count
0,10,About,2
1,10,Add Friend,12
2,10,Add to Playlist,9
3,10,Downgrade,7
4,10,Help,1


In [24]:
get_hour = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).hour, IntegerType())

In [25]:
df_new = df.withColumn('hour', get_hour(df.ts))

In [26]:
hour_counts = df_new.groupBy('userId', 'hour').agg(count('*').alias('count')).sort('userId', 'hour').toPandas()
hour_counts.head()

,userId,hour,count
0,10,0,27
1,10,1,18
2,10,2,34
3,10,3,40
4,10,4,52


In [27]:
# find the lowest timestamp overall
min_ts = df.groupby().min('ts').collect()[0]['min(ts)']

In [28]:
# create a column that shows the relative day for the entry
df_new = df.withColumn('rel_day', ((df.ts - min_ts)/(1000*60*60*24)).cast(IntegerType()))

In [29]:
day_counts = df_new.groupBy('userId', 'rel_day').agg(count('*').alias('count')).sort('userId', 'rel_day').toPandas()
day_counts = day_counts.sort_values(by=['userId', 'rel_day'], ascending=False)
day_counts.reset_index(inplace=True, drop=True)
day_counts.head()

,userId,rel_day,count
0,99,57,126
1,99,56,7
2,99,54,141
3,99,52,88
4,99,49,23


In [30]:
# create churn measure
churn = page_counts[page_counts['page'] == 'Cancellation Confirmation'].copy()
churn.drop('page', inplace=True, axis=1)
churn.rename(columns={'count': 'churn'}, inplace=True)
churn.head()

,userId,churn
33,100001,1
54,100003,1
80,100005,1
93,100006,1
104,100007,1


In [31]:
# create labels with all users
labels = users.copy()

# add churn
labels = labels.merge(churn, on='userId', how='left')
labels.fillna(0, inplace=True)
labels.churn = labels.churn.astype(int)

# set user id as the index
labels.set_index('userId', inplace=True)

print(labels.count())
labels.head()

churn    225
dtype: int64


,churn
userId,
100010,0
200002,0
125,1
51,1
124,0


In [32]:
# create features with all users
features = users.copy()

# add thumbs up
thumbs_up = page_counts[page_counts['page'] == 'Thumbs Up'][['userId', 'count']]
thumbs_up.rename(columns={'count': 'thumbs_up'}, inplace=True)
features = features.merge(thumbs_up, on='userId', how='left')

# add thumbs down
thumbs_down = page_counts[page_counts['page'] == 'Thumbs Down'][['userId', 'count']]
thumbs_down.rename(columns={'count': 'thumbs_down'}, inplace=True)
features = features.merge(thumbs_down, on='userId', how='left')

# add add to playlist
add_to_playlist = page_counts[page_counts['page'] == 'Add to Playlist'][['userId', 'count']]
add_to_playlist.rename(columns={'count': 'add_to_playlist'}, inplace=True)
features = features.merge(add_to_playlist, on='userId', how='left')

# add add friend
add_friend = page_counts[page_counts['page'] == 'Add Friend'][['userId', 'count']]
add_friend.rename(columns={'count': 'add_friend'}, inplace=True)
features = features.merge(add_friend, on='userId', how='left')

# add error
error = page_counts[page_counts['page'] == 'Error'][['userId', 'count']]
error.rename(columns={'count': 'error'}, inplace=True)
features = features.merge(error, on='userId', how='left')

# set user id as the index
features.set_index('userId', inplace=True)

# fill in any missing data
features.fillna(0, inplace=True)

print(features.count())
print(features.shape)
features.head()

thumbs_up          220
thumbs_down        203
add_to_playlist    215
add_friend         206
error              117
dtype: int64
(225, 5)


,thumbs_up,thumbs_down,add_to_playlist,add_friend,error
userId,,,,,
100010,17.0,5.0,7.0,4.0,NaN
200002,21.0,6.0,8.0,4.0,NaN
125,NaN,NaN,NaN,NaN,NaN
51,100.0,21.0,52.0,28.0,1.0
124,171.0,41.0,118.0,74.0,6.0


In [33]:
# add hour counts for each user

# set up hour windows so the hours can be grouped in buckets instead of the full 24 slots
hour_window = 3
buckets = int(24 / hour_window)

# create zero columns in features for all our hour buckets
for bucket in range(buckets):
    col_name = 'hour_' + str(bucket)
    features[col_name] = 0.0

# go through each of the hours counts we found earlier and fill them in our features
for index, row in hour_counts.iterrows():
    # find the total song counts for this user so we can use for scaling
    userId = row['userId']
    total_count = total_counts.at[userId, 'count']
    
    # find the column name to use based on the buckets
    col_index = int(row['hour'] / hour_window)
    col_name = 'hour_' + str(col_index)
    features.at[row['userId'], col_name] += row['count'] / total_count

# fill in any missing data
features.fillna(0, inplace=True)

print(features.count())
print(features.shape)
features.head()

thumbs_up          225
thumbs_down        225
add_to_playlist    225
add_friend         225
error              225
hour_0             225
hour_1             225
hour_2             225
hour_3             225
hour_4             225
hour_5             225
hour_6             225
hour_7             225
dtype: int64
(225, 13)


,thumbs_up,thumbs_down,add_to_playlist,add_friend,error,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7
userId,,,,,,,,,,,,,
100010,17.0,5.0,7.0,4.0,0.0,0.010499,0.149606,0.133858,0.141732,0.278215,0.270341,0.015748,0.000000
200002,21.0,6.0,8.0,4.0,0.0,0.050633,0.000000,0.067511,0.109705,0.122363,0.200422,0.265823,0.183544
125,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
51,100.0,21.0,52.0,28.0,1.0,0.129058,0.137987,0.109984,0.100649,0.096997,0.121347,0.155844,0.148133
124,171.0,41.0,118.0,74.0,6.0,0.110052,0.103005,0.097409,0.098653,0.131399,0.122280,0.170570,0.166632


In [34]:
# add day counts for each user

# configure the number of days we should keep
days_to_keep = 7

# go through each of the day counts we collected earlier
cur_userId = 0
cur_rel_day = 0
for index, row in day_counts.iterrows():
    userId = row['userId']
    rel_day = row['rel_day']
    
    # if we switched to a new user, reset the index
    if userId != cur_userId:
        cur_userId = userId
        day_index = 0
        
    # otherwise carry on with the same user using the relative days
    else:
        day_index += (cur_rel_day - rel_day)
        
    # update for the next loop
    cur_rel_day = rel_day

    # if we are past our days to keep, move on
    if day_index >= days_to_keep:
        continue
        
    # find the total song counts for this user so we can use for scaling
    total_count = total_counts.at[userId, 'count']
    
    # set the counts in our features
    col = 'day_' + str(day_index)
    features.at[row['userId'], col] = row['count'] / total_count
    
features.fillna(0, inplace=True)

print(features.count())
print(features.shape)
features.head()

thumbs_up          225
thumbs_down        225
add_to_playlist    225
add_friend         225
error              225
hour_0             225
hour_1             225
hour_2             225
hour_3             225
hour_4             225
hour_5             225
hour_6             225
hour_7             225
day_0              225
day_1              225
day_3              225
day_5              225
day_2              225
day_6              225
day_4              225
dtype: int64
(225, 20)


,thumbs_up,thumbs_down,add_to_playlist,add_friend,error,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,day_0,day_1,day_3,day_5,day_2,day_6,day_4
userId,,,,,,,,,,,,,,,,,,,,
100010,17.0,5.0,7.0,4.0,0.0,0.010499,0.149606,0.133858,0.141732,0.278215,0.270341,0.015748,0.000000,0.086614,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
200002,21.0,6.0,8.0,4.0,0.0,0.050633,0.000000,0.067511,0.109705,0.122363,0.200422,0.265823,0.183544,0.295359,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
125,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
51,100.0,21.0,52.0,28.0,1.0,0.129058,0.137987,0.109984,0.100649,0.096997,0.121347,0.155844,0.148133,0.049107,0.048701,0.000000,0.104302,0.069399,0.131899,0.0
124,171.0,41.0,118.0,74.0,6.0,0.110052,0.103005,0.097409,0.098653,0.131399,0.122280,0.170570,0.166632,0.000207,0.026736,0.014508,0.000000,0.011192,0.000000,0.0


# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [35]:
# normalize all the features
X = features.values
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
print(X)
print(len(X))

# get the labels
y = labels['churn'].values
print(y)
print(len(y))

[[ 0.0389016   0.06666667  0.02916667 ...,  0.          0.          0.        ]
 [ 0.04805492  0.08        0.03333333 ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 ..., 
 [ 0.12585812  0.46666667  0.125      ...,  0.          0.09045597  0.        ]
 [ 0.01601831  0.01333333  0.02083333 ...,  0.          0.          0.        ]
 [ 0.01830664  0.02666667  0.0125     ...,  0.          0.          0.        ]]
225
[0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 1 0 0
 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 1]
225


In [48]:
# split into training and test sets
target_names = ['no churn', 'churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((180, 20), (45, 20), (180,), (45,))

In [49]:
from sklearn.linear_model import LogisticRegression

lrm = LogisticRegression().fit(X_train, y_train)
y_pred = lrm.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.83      0.97      0.90        36
      churn       0.67      0.22      0.33         9

avg / total       0.80      0.82      0.78        45



(array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [50]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.83      0.94      0.88        36
      churn       0.50      0.22      0.31         9

avg / total       0.76      0.80      0.77        45



(array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [51]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.80      1.00      0.89        36
      churn       0.00      0.00      0.00         9

avg / total       0.64      0.80      0.71        45



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [52]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
y_pred = dtc.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.86      0.86      0.86        36
      churn       0.44      0.44      0.44         9

avg / total       0.78      0.78      0.78        45



(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [53]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.83      0.97      0.90        36
      churn       0.67      0.22      0.33         9

avg / total       0.80      0.82      0.78        45



(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [72]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(hidden_layer_sizes=(200,), alpha=1e-5, solver='lbfgs', max_iter=3000)
mlpc.fit(X_train, y_train)
y_pred = mlpc.predict(X_test)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f1_score(y_test, y_pred))
y_pred, y_test

             precision    recall  f1-score   support

   no churn       0.87      0.75      0.81        36
      churn       0.36      0.56      0.43         9

avg / total       0.77      0.71      0.73        45

0.434782608696


(array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [73]:
from sklearn.model_selection import GridSearchCV

# create f1 scorer to maximize churn prediction
f1 = make_scorer(f1_score, average='binary')

# perform grid search on MLP classifier
params = {
    'max_iter': [3000, 5000],
    'activation': ['logistic', 'tanh', 'relu'],
    'warm_start': [False, True],
    'hidden_layer_sizes': [(200,), (100,), (300,), (50,10), (50,20), (100,20), (200,20)]
}

mlpc = MLPClassifier(solver='lbfgs')
cv = GridSearchCV(mlpc, param_grid=params, verbose=1, scoring=f1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(cv.best_params_)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f1_score(y_test, y_pred))
y_pred, y_test

Fitting 3 folds for each of 84 candidates, totalling 252 fits


[Parallel(n_jobs=1)]: Done 252 out of 252 | elapsed:  1.7min finished


{'activation': 'logistic', 'hidden_layer_sizes': (200, 20), 'max_iter': 3000, 'warm_start': False}
             precision    recall  f1-score   support

   no churn       0.89      0.86      0.87        36
      churn       0.50      0.56      0.53         9

avg / total       0.81      0.80      0.80        45

0.526315789474


(array([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [74]:
from sklearn.model_selection import GridSearchCV

# create f1 scorer to maximize churn prediction
f1 = make_scorer(f1_score, average='binary')

# perform grid search on Decision Tree
params = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
}

dtc = DecisionTreeClassifier()
cv = GridSearchCV(dtc, param_grid=params, verbose=1, scoring=f1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(cv.best_params_)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f1_score(y_test, y_pred))
y_pred, y_test

Fitting 3 folds for each of 4 candidates, totalling 12 fits
{'criterion': 'gini', 'splitter': 'best'}
             precision    recall  f1-score   support

   no churn       0.86      0.83      0.85        36
      churn       0.40      0.44      0.42         9

avg / total       0.77      0.76      0.76        45

0.421052631579


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.0s finished


(array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

In [75]:
from sklearn.model_selection import GridSearchCV

# create f1 scorer to maximize churn prediction
f1 = make_scorer(f1_score, average='binary')

# perform grid search on Decision Tree
params = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': [100, 300, 500],
    'bootstrap': [False, True]
}

rfc = RandomForestClassifier()
cv = GridSearchCV(rfc, param_grid=params, verbose=1, scoring=f1)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print(cv.best_params_)
print(classification_report(y_test, y_pred, target_names=target_names))
print(f1_score(y_test, y_pred))
y_pred, y_test

Fitting 3 folds for each of 12 candidates, totalling 36 fits
{'bootstrap': False, 'criterion': 'gini', 'n_estimators': 100}
             precision    recall  f1-score   support

   no churn       0.85      0.94      0.89        36
      churn       0.60      0.33      0.43         9

avg / total       0.80      0.82      0.80        45

0.428571428571


[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:   18.0s finished


(array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.